# Requirements

In [1]:
!pip3 install pymystem3==0.1.10 --user
!pip3 install pymorphy2[fast] --user
!pip3 install razdel --user
!pip3 install gensim --user
!pip3 install nltk --user
!pip3 install rusenttokenize --user
!pip3 install regex --user

  Using cached https://files.pythonhosted.org/packages/45/1e/0c169c6a5381e241ba7404532c16a21d86ab872c9bed8bdcd4c423954103/requests-2.24.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/56/aa/4ef5aa67a9a62505db124a5cb5262332d1d4153462eb8fd89c9fa41e5d92/urllib3-1.25.11-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/c4/6c4fe722df5343c33226f0b4e0bb042e4dc13483228b4718baf286f86d87/certifi-2020.6.20-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl
    100% |████████████████████████████████| 61kB 659kB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/3a/79/bea0021eeb7eeefde22ef9e96badf174068a2dd20264b9a378f2be1cdd9e/pymorphy2_dicts_ru-2

# Imports

In [93]:
import re
import os
import json
import string
from string import punctuation
from collections import Counter


from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from rusenttokenize import ru_sent_tokenize
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

# Constant

In [94]:
mystem = Mystem()
morph = MorphAnalyzer()
stemmer = SnowballStemmer('russian')

# Data load

In [55]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/alex/.wget-hsts'. HSTS will be disabled.
--2020-11-05 20:52:11--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1965598 (1.9M) [text/plain]
Saving to: ‘zhivago.txt’

zhivago.txt         100%[===================>]   1.87M  7.18MB/s    in 0.3s    

2020-11-05 20:52:11 (7.18 MB/s) - ‘zhivago.txt’ saved [1965598/1965598]



In [95]:
!ls

Untitled.ipynb	hw1.ipynb  zhivago.txt


In [106]:
text = open('zhivago.txt').read()

In [107]:
# Show first 100 symbols
text[:100]

'<author><first-name>Борис</first-name><middle-name>Леонидович</middle-name><last-name>Пастернак</las'

# Task 1

In [108]:
text = re.sub('\n', ' ', text)
text = re.sub('<binary.*?</binary>', ' ', text)
text = re.sub('<nickname.*?</nickname>', ' ', text)
text = re.sub('<program-used.*?</program-used>', ' ', text)
text = re.sub('<src-url.*?</src-url>', ' ', text)
text = re.sub('<date.*?</date>', ' ', text)
text = re.sub('<id.*?</id>', ' ', text)
text = re.sub('<lang.*?</lang>', ' ', text)
text = re.sub('<version.*?</version>', ' ', text)
text = re.sub('<.*?>', ' ', text)
text = re.sub(u'\xa0', ' ', text)
text = re.sub(' {2,}', ' ', text)

In [109]:
len(text)

1020144

In [110]:
text[:1000]

' Борис Леонидович Пастернак Доктор Живаго «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни. «Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора. Человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть. Роман Пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости. Перечитывать «Доктора Живаго» стоит именно тогда, когда кажется, что жить не стоит. Тогда десять строк из этого романа могут сделать то же, что делает любовь 

# Task 2

In [111]:
text = text.lower()

remove = string.punctuation
remove += '«»—…“”'
remove = remove.replace('.', '')

print('Symbols to remove', remove)

#remove = dict.fromkeys(map(ord, remove), '')
#text = text.translate(remove).strip()

text = ' '.join([word.strip(remove) for word in text.split(' ')])

Symbols to remove !"#$%&'()*+,-/:;<=>?@[\]^_`{|}~«»—…“”


In [112]:
text[:2000]

' борис леонидович пастернак доктор живаго доктор живаго  итоговое произведение бориса пастернака книга всей его жизни. этот роман принес его автору мировую известность и нобелевскую премию присуждение которой обернулось для поэта оголтелой политической травлей обвинениями в измене родине и в результате стоило ему жизни. доктор живаго  роман сама ткань которого убедительнее свидетельствует о чуде чем все размышления доктора и обобщения автора. человек который так пишет бесконечно много пережил и передумал и главные его чувства на свете  восхищенное умиление и слезное сострадание конечно есть в его мире место и презрению и холодному отстранению  но не в них суть. роман пастернака  оплакивание прежних заблуждений и их жертв те кто не разделяет молитвенного восторга перед миром достойны прежде всего жалости. перечитывать доктора живаго стоит именно тогда когда кажется что жить не стоит. тогда десять строк из этого романа могут сделать то же что делает любовь в одном из стихотворений докто

In [113]:
sentences = ru_sent_tokenize(text)

In [114]:
# поиск повторяющихся предложений
c = Counter(sentences)
c.most_common(10)

[('да.', 10),
 ('– хорошо.', 6),
 ('– знаю.', 4),
 ('сеялки.', 4),
 ('молотилки».', 4),
 ('но дело не в этом.', 3),
 ('слушай.', 3),
 ('– еще бы.', 3),
 ('– да.', 3),
 ('– я знаю.', 3)]

Выгляит абсолютном логичнм результат. Предложения только согласия встречается больше всего раз.

In [115]:
sentences = [[token.text for token in list(razdel_tokenize(sent))] for sent in sentences]

In [116]:
sentences[2]

['доктор',
 'живаго',
 'роман',
 'сама',
 'ткань',
 'которого',
 'убедительнее',
 'свидетельствует',
 'о',
 'чуде',
 'чем',
 'все',
 'размышления',
 'доктора',
 'и',
 'обобщения',
 'автора',
 '.']

In [117]:
# поиск самого частотного токена больше длины 6
acceptable_tokens = list()
for sent in sentences:
    for token in sent:
        if len(token) > 6:
            acceptable_tokens.append(token)

c = Counter(acceptable_tokens)
c.most_common(1)

[('андреевич', 289)]

# Task 3

In [118]:
# простемим все токены и посмотрим на самые частотные
acceptable_tokens = list()
for sent in sentences:
    for token in sent:
        tmp_stem = stemmer.stem(token)
        if token != tmp_stem:
            acceptable_tokens.append(tmp_stem)

c = Counter(acceptable_tokens)
c.most_common(40)

[('эт', 1924),
 ('он', 1243),
 ('был', 1200),
 ('ег', 1085),
 ('е', 781),
 ('котор', 634),
 ('юр', 562),
 ('сво', 559),
 ('друг', 458),
 ('говор', 391),
 ('так', 372),
 ('ил', 367),
 ('когд', 364),
 ('ещ', 352),
 ('как', 347),
 ('сам', 344),
 ('одн', 341),
 ('себ', 335),
 ('не', 326),
 ('чтоб', 324),
 ('ем', 319),
 ('теб', 294),
 ('мен', 293),
 ('тольк', 293),
 ('жизн', 283),
 ('больш', 283),
 ('врем', 267),
 ('рук', 266),
 ('лар', 257),
 ('дом', 250),
 ('тепер', 242),
 ('над', 241),
 ('сказа', 241),
 ('мо', 240),
 ('нег', 235),
 ('пот', 221),
 ('дел', 218),
 ('мест', 211),
 ('стал', 200),
 ('част', 200)]

In [119]:
# кажется можно посмотреть на исхожные формы след. стемов: мен, стал

result = {
    'мен': set(),
    'стал': set()
}

acceptable_tokens = list()
for sent in sentences:
    for token in sent:
        tmp_stem = stemmer.stem(token)
        if tmp_stem in result:
            result[tmp_stem].add(token)

In [120]:
result

{'мен': {'мена', 'мене', 'менее', 'меня'},
 'стал': {'стал', 'стала', 'стали', 'стало', 'сталось', 'сталь'}}

Вывод:

кажется, что я угадал. Стем "стал" приводит слова стал и сталь (абсолютно разные во всех смыслах) к одному. А также "мен" содержит в себе слова "мена", "менее" и "меня", и они тоже являются тремя разными словами. 

In [125]:
# простемим все токены и посмотрим на те, которые не изменились
acceptable_tokens = list()
for sent in sentences:
    for token in sent:
        tmp_stem = stemmer.stem(token)
        if token == tmp_stem and len(tmp_stem) > 4:
            acceptable_tokens.append(tmp_stem)

c = Counter(acceptable_tokens)
c.most_common(1)

[('андреевич', 289)]

Резултат похож на поиск самого частотного токена больше длины 6. Тут snowball не отработал, но при этом слово является одним из самых частых.

# Task 4

In [88]:
stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Предлагаю также добавить в стоп. слова:
- нибудь | т.к. часть слова может остаться при неправильной пред. обработки
- это | т.к. частица не несет смысла
- также | т.к. союз и не несет смысла 
- либо | т.к. союз и не несет смысла 
- ко | и в целом все предлоги с "о" которых нет в списке стоп-слов. 

# Task 5

In [89]:
acceptable_tokens = list()
for sent in sentences:
    for token in sent:
        morph_token = morph.parse(token)[0].normal_form
        mystem_token = mystem.lemmatize(token)[0]
        if morph_token != mystem_token:
            tmp_str = f'token:{token},morph:{morph_token},mystem:{mystem_token}'
            acceptable_tokens.append(tmp_str)

In [90]:
c = Counter(acceptable_tokens)
c.most_common(50)

[('token:–,morph:–,mystem:–\n', 2421),
 ('token:все,morph:всё,mystem:все', 761),
 ('token:еще,morph:ещё,mystem:еще', 352),
 ('token:со,morph:с,mystem:со', 235),
 ('token:во,morph:в,mystem:во', 207),
 ('token:чем,morph:чем,mystem:что', 204),
 ('token:»,morph:»,mystem:»\n', 198),
 ('token:может,morph:мочь,mystem:может', 162),
 ('token:ним,morph:они,mystem:он', 161),
 ('token:больше,morph:большой,mystem:больше', 142),
 ('token:того,morph:тот,mystem:то', 133),
 ('token:есть,morph:есть,mystem:быть', 117),
 ('token:всех,morph:весь,mystem:все', 114),
 ('token:тем,morph:тем,mystem:то', 108),
 ('token:стал,morph:стать,mystem:становиться', 103),
 ('token:дома,morph:дом,mystem:дома', 97),
 ('token:всем,morph:весь,mystem:все', 97),
 ('token:об,morph:о,mystem:об', 88),
 ('token:стало,morph:стать,mystem:становиться', 85),
 ('token:том,morph:тот,mystem:том', 76),
 ('token:всего,morph:весь,mystem:всего', 72),
 ('token:стали,morph:стать,mystem:становиться', 67),
 ('token:свете,morph:свет,mystem:света',

В примерах ниже видно, что pymorphy2 приводит все к мужскому роду, хотя в контексте книги это могут быть разные персонажи
- ('token:федоровна,morph:фёдорович,mystem:федоровна', 45)
- ('token:антипова,morph:антипов,mystem:антипова', 30),

Буква Ё:
- ('token:все,morph:всё,mystem:все', 761),
- ('token:еще,morph:ещё,mystem:еще', 352),

Т.е. pymorphy2 добавляет букву ё даже когда ее нет в тексте, что считаю неправильным

Удаление буквы о в предлогах у pymorphy2. Это также считаю проблемой так как это уже разные токены.
- ('token:во,morph:в,mystem:во', 207),
- ('token:ко,morph:к,mystem:ко', 29),

Неуместное приведение к множественному числу у pymorphy2 :
- ('token:ним,morph:они,mystem:он', 161),

Вывод: для обработки данноо текста лучше использовать mystem